In [1]:
import pandas as pd
import pyspark.sql.functions as F
from datetime import datetime
from pyspark.sql.types import *
from pyspark import StorageLevel

import numpy as np
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.set_option("mode.chained_assignment", None)

In [2]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
# from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler

from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

In [3]:
# !pip install scikit-plot

In [4]:
import sklearn
import scikitplot as skplt
from sklearn.metrics import classification_report, confusion_matrix, precision_score

<hr />
<hr />
<hr />

In [5]:
result_schema = StructType([
                    StructField('experiment_filter', StringType(), True),
                    StructField('undersampling_method', StringType(), True),
                    StructField('undersampling_column', StringType(), True),
                    StructField('filename', StringType(), True),
                    StructField('experiment_id', StringType(), True),
                    StructField('n_covid', IntegerType(), True),
                    StructField('n_not_covid', IntegerType(), True),
                    StructField('model_name', StringType(), True),
                    StructField('model_seed', StringType(), True),
                    StructField('model_maxIter', IntegerType(), True),
                    StructField('model_maxDepth', IntegerType(), True),
                    StructField('model_maxBins', IntegerType(), True),
                    StructField('model_minInstancesPerNode', IntegerType(), True),
                    StructField('model_minInfoGain', FloatType(), True),
                    StructField('model_featureSubsetStrategy', StringType(), True),
                    StructField('model_n_estimators', IntegerType(), True),
                    StructField('model_learning_rate', FloatType(), True),
                    StructField('model_impurity', StringType(), True),
                    StructField('model_AUC_ROC', StringType(), True),
                    StructField('model_AUC_PR', StringType(), True),
                    StructField('model_covid_precision', StringType(), True),
                    StructField('model_covid_recall', StringType(), True),
                    StructField('model_covid_f1', StringType(), True),
                    StructField('model_not_covid_precision', StringType(), True),
                    StructField('model_not_covid_recall', StringType(), True),
                    StructField('model_not_covid_f1', StringType(), True),
                    StructField('model_avg_precision', StringType(), True),
                    StructField('model_avg_recall', StringType(), True),
                    StructField('model_avg_f1', StringType(), True),
                    StructField('model_avg_acc', StringType(), True),
                    StructField('model_TP', StringType(), True),
                    StructField('model_TN', StringType(), True),
                    StructField('model_FN', StringType(), True),
                    StructField('model_FP', StringType(), True),
                    StructField('model_time_exec', StringType(), True),
                    StructField('model_col_set', StringType(), True)
                          ])

<hr />
<hr />
<hr />

In [6]:
# undersamp_col = ['03-STRSAMP-AG', '04-STRSAMP-EW'] 
# dfs = ['ds-1', 'ds-2', 'ds-3']
# cols_sets = ['cols_set_1', 'cols_set_2', 'cols_set_3']

undersamp_col = ['04-STRSAMP-EW']
dfs = ['ds-1']
cols_sets = ['cols_set_1']

In [7]:
# lists of params
model_maxIter = [20, 50, 100] 
model_maxDepth = [3, 5, 7] 
model_maxBins = [32, 64] 
# model_learningRate = [0.01, 0.1, 0.5]
# model_loss = ['logLoss', 'leastSquaresError', 'leastAbsoluteError']


list_of_param_dicts = []

for maxIter in model_maxIter:
    for maxDepth in model_maxDepth:
        for maxBins in model_maxBins: 
            params_dict = {}
            params_dict['maxIter'] = maxIter
            params_dict['maxDepth'] = maxDepth
            params_dict['maxBins'] = maxBins
            list_of_param_dicts.append(params_dict)

            

print("There is {} set of params.".format(len(list_of_param_dicts)))
# list_of_param_dicts

There is 18 set of params.


In [8]:
prefix = 'gs://ai-covid19-datalake/trusted/experiment_map/'

<hr />
<hr />
<hr />

In [9]:
# filename = 'gs://ai-covid19-datalake/trusted/experiment_map/03-STRSAMP-AG/ds-1/cols_set_1/experiment0.parquet'
# df = spark.read.parquet(filename)
# df.limit(2).toPandas()

In [10]:
# params_dict = {'maxIter': 100,
#                'maxDepth': 3,
#                'maxBins': 32,
#                'learningRate': 0.5,
#                'loss': 'leastAbsoluteError'}
# cols = 'cols_set_1'
# experiment_filter = 'ds-1'
# undersampling_method = '03-STRSAMP-AG', 
# experiment_id = 0

In [11]:
# run_gbt(df, params_dict, cols, filename, experiment_filter, undersampling_method, experiment_id)

<hr />
<hr />
<hr />

In [12]:
def run_gbt(exp_df, params_dict, cols, filename, experiment_filter, 
            undersampling_method, experiment_id):
    import time
    start_time = time.time()
    
    n_covid = exp_df.filter(F.col('CLASSI_FIN') == 1.0).count()
    n_not_covid = exp_df.filter(F.col('CLASSI_FIN') == 0.0).count()
    
    
    id_cols = ['NU_NOTIFIC', 'CLASSI_FIN']

    labelIndexer = StringIndexer(inputCol="CLASSI_FIN", outputCol="indexedLabel").fit(exp_df)    
    
    input_cols = [x for x in exp_df.columns if x not in id_cols]
    assembler = VectorAssembler(inputCols = input_cols, outputCol= 'features')
    exp_df = assembler.transform(exp_df)
    
    # Automatically identify categorical features, and index them.
    # Set maxCategories so features with > 4 distinct values are treated as continuous.
    featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=30).fit(exp_df)
    
    # Split the data into training and test sets (30% held out for testing)
    (trainingData, testData) = exp_df.randomSplit([0.7, 0.3])
    trainingData = trainingData.persist(StorageLevel.MEMORY_ONLY)
    testData = testData.persist(StorageLevel.MEMORY_ONLY)
    
    # Train a RandomForest model.
    gbt = GBTClassifier(labelCol = "indexedLabel", featuresCol = "indexedFeatures",
                       maxIter = params_dict['maxIter'],
                       maxDepth = params_dict['maxDepth'],
                       maxBins = params_dict['maxBins'])
    
    # Convert indexed labels back to original labels.
    labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

    # Chain indexers and forest in a Pipeline
    pipeline = Pipeline(stages=[labelIndexer, featureIndexer, gbt, labelConverter])

    # Train model.  This also runs the indexers.
    model = pipeline.fit(trainingData)
    
    # Make predictions.
    predictions = model.transform(testData)    
    
    
    pred = predictions.select(['CLASSI_FIN', 'predictedLabel'])\
                  .withColumn('predictedLabel', F.col('predictedLabel').cast('double'))\
                  .withColumn('predictedLabel', F.when(F.col('predictedLabel') == 1.0, 'covid').otherwise('n-covid'))\
                  .withColumn('CLASSI_FIN', F.when(F.col('CLASSI_FIN') == 1.0, 'covid').otherwise('n-covid'))\
                  .toPandas()

    y_true = pred['CLASSI_FIN'].tolist()
    y_pred = pred['predictedLabel'].tolist()
    
    report = classification_report(y_true, y_pred, output_dict=True)
    
    
    evaluator_ROC = BinaryClassificationEvaluator(labelCol="indexedLabel", rawPredictionCol="prediction", metricName="areaUnderROC")
    accuracy_ROC = evaluator_ROC.evaluate(predictions)


    
    evaluator_PR = BinaryClassificationEvaluator(labelCol="indexedLabel", rawPredictionCol="prediction", metricName="areaUnderPR")
    accuracy_PR = evaluator_PR.evaluate(predictions)
    
    conf_matrix = confusion_matrix(y_true, y_pred)

    result_dict = {}
    
    result_dict['experiment_filter'] = experiment_filter
    result_dict['undersampling_method'] = undersampling_method
    result_dict['filename'] = filename
    result_dict['experiment_id'] = experiment_id
    result_dict['n_covid'] = n_covid
    result_dict['n_not_covid'] = n_not_covid
    result_dict['model_name'] = 'GBT'
    result_dict['params'] = params_dict
    result_dict['model_AUC_ROC'] = accuracy_ROC
    result_dict['model_AUC_PR'] = accuracy_PR
    result_dict['model_covid_precision'] = report['covid']['precision']
    result_dict['model_covid_recall'] = report['covid']['recall']
    result_dict['model_covid_f1'] = report['covid']['f1-score']
    result_dict['model_not_covid_precision'] = report['n-covid']['precision']
    result_dict['model_not_covid_recall'] = report['n-covid']['recall']
    result_dict['model_not_covid_f1'] = report['n-covid']['f1-score']
    result_dict['model_avg_precision'] = report['macro avg']['precision']
    result_dict['model_avg_recall'] = report['macro avg']['recall']
    result_dict['model_avg_f1'] = report['macro avg']['f1-score']
    result_dict['model_avg_acc'] = report['accuracy']
    result_dict['model_TP'] = conf_matrix[0][0]
    result_dict['model_TN'] = conf_matrix[1][1]
    result_dict['model_FN'] = conf_matrix[0][1]
    result_dict['model_FP'] = conf_matrix[1][0]
    result_dict['model_time_exec'] = time.time() - start_time
    result_dict['model_col_set'] = cols
    
    return result_dict

<hr />
<hr />
<hr />

# Running GBT on 10 samples for each experiment
### 3x col sets -> ['cols_set_1', 'cols_set_2', 'cols_set_3']
### 3x model_maxIter -> [100, 200, 300]
### 3x model_maxDepth -> [5, 10, 15]
### 3x model_maxBins -> [16, 32, 64]
Total: 10 * 3 * 3 * 3 * 3 = 810

In [13]:
experiments = []

### Datasets: strat_samp_lab_agegrp

In [ ]:
for uc in undersamp_col: 
    for ds in dfs:
        for col_set in cols_sets:
            for params_dict in list_of_param_dicts: 
                for id_exp in range(50):
                    filename = prefix + uc + '/' + ds + '/' + col_set + '/' + 'experiment' + str(id_exp) + '.parquet'
                    exp_dataframe = spark.read.parquet(filename)
                    print('read {}'.format(filename))
                    
                    undersampling_method = uc
                    experiment_filter = ds
                    experiment_id = id_exp

                    try:                     
                        model = run_gbt(exp_dataframe, params_dict, col_set, filename, experiment_filter, undersampling_method, experiment_id)
                        experiments.append(model)

                        print("Parameters ==> {}\n Results: \n AUC_PR: {} \n Precision: {} \n Time: {}".format(str(params_dict), str(model['model_AUC_PR']), str(model['model_avg_precision']), str(model['model_time_exec'])))
                        print('=========================== \n')
                    except:
                        print('=========== W A R N I N G =========== \n')
                        print('Something wrong with the exp: {}, {}, {}'.format(filename, params_dict, col_set))

read gs://ai-covid19-datalake/trusted/experiment_map/04-STRSAMP-EW/ds-1/cols_set_1/experiment0.parquet
Parameters ==> {'maxIter': 20, 'maxDepth': 3, 'maxBins': 32}
 Results: 
 AUC_PR: 0.9132019390612496 
 Precision: 0.9427864467480684 
 Time: 27.99976134300232

read gs://ai-covid19-datalake/trusted/experiment_map/04-STRSAMP-EW/ds-1/cols_set_1/experiment1.parquet
Parameters ==> {'maxIter': 20, 'maxDepth': 3, 'maxBins': 32}
 Results: 
 AUC_PR: 0.9068379457975604 
 Precision: 0.9404861772650066 
 Time: 16.83842658996582

read gs://ai-covid19-datalake/trusted/experiment_map/04-STRSAMP-EW/ds-1/cols_set_1/experiment2.parquet
Parameters ==> {'maxIter': 20, 'maxDepth': 3, 'maxBins': 32}
 Results: 
 AUC_PR: 0.9110303591309337 
 Precision: 0.9425576970807757 
 Time: 16.568068504333496

read gs://ai-covid19-datalake/trusted/experiment_map/04-STRSAMP-EW/ds-1/cols_set_1/experiment3.parquet
Parameters ==> {'maxIter': 20, 'maxDepth': 3, 'maxBins': 32}
 Results: 
 AUC_PR: 0.9100377109877157 
 Precisio

<hr />
<hr />
<hr />

In [ ]:
for i in range(len(experiments)):
    for d in list(experiments[i].keys()):
        experiments[i][d] = str(experiments[i][d])

In [ ]:
# experiments

In [ ]:
cols = ['experiment_filter', 'undersampling_method', 'filename', 'experiment_id', 'n_covid', 'n_not_covid', 'model_name', 'params', 'model_AUC_ROC', 'model_AUC_PR', 'model_covid_precision', 'model_covid_recall', 'model_covid_f1', 'model_not_covid_precision', 'model_not_covid_recall', 'model_not_covid_f1', 'model_avg_precision', 'model_avg_recall', 'model_avg_f1', 'model_avg_acc', 'model_TP', 'model_TN', 'model_FN', 'model_FP', 'model_time_exec', 'model_col_set']

In [ ]:
intermed_results = spark.createDataFrame(data=experiments).select(cols)
intermed_results.toPandas()

/usr/lib/spark/python/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


In [ ]:
intermed_results.write.parquet('gs://ai-covid19-datalake/trusted/intermed_results/STRSAMP/GBT_experiments-EW-ds1-cs1.parquet', mode='overwrite')

In [ ]:
print('finished')

In [23]:
intermed_results.show()

+-----------------+--------------------+--------------------+-------------+-------+-----------+----------+--------------------+------------------+------------------+---------------------+------------------+------------------+-------------------------+----------------------+------------------+-------------------+------------------+------------------+------------------+--------+--------+--------+--------+------------------+-------------+
|experiment_filter|undersampling_method|            filename|experiment_id|n_covid|n_not_covid|model_name|              params|     model_AUC_ROC|      model_AUC_PR|model_covid_precision|model_covid_recall|    model_covid_f1|model_not_covid_precision|model_not_covid_recall|model_not_covid_f1|model_avg_precision|  model_avg_recall|      model_avg_f1|     model_avg_acc|model_TP|model_TN|model_FN|model_FP|   model_time_exec|model_col_set|
+-----------------+--------------------+--------------------+-------------+-------+-----------+----------+--------------